# Convolutional neural network retrieval

Convolutional neural networks (CNNs) make it easy to leverage the additional structural information in the spatial retrieval input data for retrieving precipitation. This model demonstrates this using a basic UNet model.

## The training data
The SPR dataset comes in two input-data formats: *tabular* and *spatial*. The term *spatial* is used here to denote input data in 2D or image structure.
The ``ipwgml.pytorch.dataset.SPRSpatial`` implements the PyTorch dataset interface for loading SPR data in spatial format.

For the CNN-based retrieval we use the gridded geometry. Apart from that we choose retrieval input and target configuration similar to those in the fully-connected NN example.

In [1]:
%load_ext autoreload
%autoreload 2
from ipwgml.input import GMI, Ancillary, Geo, GeoIR
from ipwgml.target import TargetConfig

target_config = TargetConfig(min_rqi=0.5)
inputs = [GMI(normalize="minmax", nan=-1.5)]
geometry = "gridded"

With these settings, we can instantiate the training data dataset. By setting ``stack=True`` we also tell the retrieval to stack all input tensors instead of loading the input data as a dictionary.

In [2]:
from torch.utils.data import DataLoader
from ipwgml.pytorch.datasets import SPRSpatial

batch_size = 32
training_data = SPRSpatial(
    sensor="gmi",
    geometry=geometry,
    split="training",
    retrieval_input=inputs,
    target_config=target_config,
    stack=True,
    download=True,
)
training_loader = DataLoader(
    training_data,
    shuffle=True,
    batch_size=batch_size,
    num_workers=32,
    worker_init_fn=training_data.worker_init_fn,
    pin_memory=True
)

[06/23/24 21:56:47] INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=796060;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=753180;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

[06/23/24 21:56:48] INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=416186;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=584207;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

With the above configuration, the training data loaded by the ``training_loader`` has the following dimensions.

In [14]:
import xarray as xr
target_data = xr.load_dataset(training_data.target[0])
target_time = target_data.time
%timeit inputs[0].load_data(training_data.gmi[1], target_time)

213 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit inputs[0].load_data(training_data.gmi[1], target_time)

17.3 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
inpt, target = next(iter(training_loader))
print("Input tensor shape: ", inpt.shape)
print("Target tensor shape: ", target.shape)

Input tensor shape:  torch.Size([32, 26, 256, 256])
Target tensor shape:  torch.Size([32, 256, 256])


For the validation, we instantiate dataset and dataloader with the same settings but choose the ``validation`` split instead of the ``training`` split.

In [19]:
validation_data = SPRSpatial(
    sensor="gmi",
    geometry=geometry,
    split="validation",
    retrieval_input=inputs,
    target_config=target_config,
    augment=False,
    stack=True,
    download=True,
)
validation_loader = DataLoader(
    validation_data,
    shuffle=False,
    batch_size=32,
    num_workers=8,
    worker_init_fn=validation_data.worker_init_fn
)

[06/23/24 22:03:02] INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=364536;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=924116;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

                    INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=729562;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=708578;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

## UNet implementation

We begin by defining the convolution block that we use to build the UNet. We choose a simple ResNet block consisting of two convolutions, each followed by normalization layer and activation function and a skip connection connecting the input to the first layer with the output of the second.

In [20]:
from typing import Callable, List
import torch
from torch import nn

class ResNetBlock(nn.Module):
    """
    Implments a basic ResNet block consisting of two convolutions each
    followed by a normalization and activation layer.
    """
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        activation_fn: Callable[[], nn.Module] = nn.ReLU,
        normalization_layer: Callable[[int], nn.Module] = nn.BatchNorm2d
    ):
        """
        Args:
            in_channels: The number of channels in the input tensor.
            out_channels: The number of channels within the layer and in the output tensor.
        """
        super().__init__()
        padding = kernel_size // 2
        self.body = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
        self.projection = None
        if in_channels != out_channels:
            self.projection = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Propagate input through block.
        """
        y = self.body(x)
        if self.projection is None:
            shortcut = x
        else:
            shortcut = self.projection(x)
        return y + shortcut
      

In [21]:
from typing import Dict

class UNet(nn.Module):
    """
    UNet encoder-decoder architecture for precipitation retrievals.

    The model provides scalar estimates of the 'surface_precip' as well as probabilistic estimates of the
    'probability_of_precip' and 'probability_of_heavy_precip'.

    The model has the following components:
        - Stem: The stem is applied directly to the input and maps the number of input features to
              the features of the first encoder stage.
        - Encoder: Applied to the output from the stem. Consists of multiple stages and performs 2x
              downsampling at the beginning of each stage.
        - Decoder: Each stage consists of bilinear upsampling followed by a convolution block
              that merges the upsampled features with the output from the corresponding encoder
              layer.
        - Heads: A separate head for the retrieval outputs each consisting of a single ResNetBlock
              followed by a fully-connected output layer.
    """
    def __init__(
        self,
        input_features: int,
        internal_features: List[int],
        **block_kwargs
    ):
        """
        Args:
            input_features: The number of input features.
            internal_features: A list containing the number of features/channels within each stage
                of the encoder.
            block_kwargs: Keyword arguments to forward to ResNetBlock factory.
        """
        super().__init__()

        self.stem = ResNetBlock(input_features, internal_features[0])
        chans_in = internal_features[0]
        encoder_stages = []
        for n_features in internal_features:
            encoder_stages.append(nn.Sequential(
                nn.MaxPool2d(kernel_size=2, stride=2),
                ResNetBlock(chans_in, n_features, **block_kwargs),
            ))
            chans_in = n_features
        self.encoder = nn.ModuleList(encoder_stages)
        
        decoder_stages = []
        for n_features in internal_features[-2::-1]:
            decoder_stages.append(nn.Sequential(
                ResNetBlock(chans_in + n_features, n_features, **block_kwargs),
            ))
            chans_in = n_features
        decoder_stages.append(ResNetBlock(n_features + n_features, n_features, **block_kwargs))
            
        self.decoder = nn.ModuleList(decoder_stages)
        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear")
        
        heads = {}
        for output in ["surface_precip", "probability_of_precip", "probability_of_heavy_precip"]:
            heads[output] = nn.Sequential(
                ResNetBlock(n_features, n_features, kernel_size=1, **block_kwargs),
                nn.Conv2d(n_features, 1, kernel_size=1)
            )
        self.heads = nn.ModuleDict(heads)

    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Propagate input through network.

        Args:
            x: The tensor containing the input data.

        Return:
            A dictionary containing mapping the keys 'surface_precip', 'probability_of_precip',
            'probability_of_heavy_precip' to the corresponding retrieval results.
        """
        y = self.stem(x)
        shortcuts = []
        for layer in self.encoder:
            shortcuts.append(y)
            y = layer(y)

        shortcuts.reverse()
        for shortcut, layer in zip(shortcuts, self.decoder):
            y = self.upsample(y)
            y = torch.cat([y, shortcut], dim=1)
            y = layer(y)

        return {
            name: head(y) for name, head in self.heads.items()
        }
            

Based on the UNet model defined above, we define a lightning module to manage the training of the model.

The model is trained using MSE loss for the ``surface_precipitation`` output and binary cross-entropy loss for the ``probability_of_precip`` and ``probability_of_heavy_precip`` outputs. Moreover, we use the Adam optimizer with a cosine-annealing learning rate schedule.

In [22]:
from typing import Any, Callable, Dict

import torch
from torch import optim
from torch import nn
from torch.nn.functional import binary_cross_entropy_with_logits
import lightning as L

OUTPUTS = [
    "surface_precip",
    "probability_of_precipitation",
    "probability_of_heavy_precipitation"
]

class IPWGUNet(L.LightningModule):
    """
    Lightning module implementing a multi-layer perceptron (MLP) for retrieving precipitation from satellite
    observations.
    """
    def __init__(
        self,
        input_features: int,
        internal_features: List[int],
        n_epochs: int = 20,
        **block_kwargs
    ):
        """
        Args:
            n_input_features: The number of features in the input
            n_hidden_layers: The number of hidden layers in the MLP
            n_neurons: The number of neurons in the hidden layers
            activation_fn: A callable to create activation function layers.
            normalization_layer: A callable to create normalization layers.
            n_epochs: The numebr of epochs the model will be trained for.
        """
        super().__init__()
        self.n_epochs = n_epochs
        self.model = UNet(input_features, internal_features)

    def forward(self, retrieval_input: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Forward retrieval input through network and produce dictionary with predictions.

        Args:
            retrieval_input: The retrieval input as a single torch.Tensor.

        Return:
            A dictionary containing the predictions for 'surface_precip', 'probability_of_precipitation',
            and 'probability_of_heavy_precipitation'.
        """
        return self.model(retrieval_input)
        
    def training_step(self, batch, batch_idx) -> torch.Tensor:
        """
        Calculates the loss-function gradients for the MLP.

        The loss is calculated as the sum of the MSE for 'surface_precip' and the binary cross-entropy loss
        for precipitation detection and heavy precipitation detection.

        Args:
            batch: A tuple containing the training data loaded from the data loader.
            batch_idx: The index of the batch in the current epoch. Not used.

        Return:
            A scalar torch.Tensor containing the total loss.
        """
        inpt, surface_precip = batch
        pred = self(inpt)

        valid = torch.isfinite(surface_precip)
        surface_precip = surface_precip[valid]
        precip_mask = (surface_precip > 1e-3).to(dtype=torch.float32)
        heavy_precip_mask = (surface_precip > 10).to(dtype=torch.float32)
        surface_precip_pred = pred["surface_precip"][:, 0][valid]
        pop = pred["probability_of_precip"][:, 0][valid]
        pohp = pred["probability_of_heavy_precip"][:, 0][valid]
        
        # MSE loss for QPE
        loss_estim = ((surface_precip_pred - surface_precip) ** 2).mean()
        # BCE loss for detection targets
        loss_detect = binary_cross_entropy_with_logits(pop, precip_mask)
        loss_detect_heavy = binary_cross_entropy_with_logits(pohp, heavy_precip_mask)
        tot_loss =  loss_estim + loss_detect + loss_detect_heavy
        return tot_loss

    def validation_step(self, batch, batch_idx) -> None:
        """
        Calculates the loss-function values on validation data.

        Args:
            batch: A tuple containing the training data loaded from the data loader.
            batch_idx: The index of the batch in the current epoch. Not used.
        """
        inpt, surface_precip = batch
        pred = self(inpt)

        valid = torch.isfinite(surface_precip)
        surface_precip = surface_precip[valid]
        precip_mask = (surface_precip > 1e-3).to(dtype=torch.float32)
        heavy_precip_mask = (surface_precip > 10).to(dtype=torch.float32)
        surface_precip_pred = pred["surface_precip"][:, 0][valid]
        pop = pred["probability_of_precip"][:, 0][valid]
        pohp = pred["probability_of_heavy_precip"][:, 0][valid]
        
        # MSE loss for QPE
        loss_estim = ((surface_precip_pred - surface_precip) ** 2).mean()
        # BCE loss for detection targets
        loss_detect = binary_cross_entropy_with_logits(pop, precip_mask)
        loss_detect_heavy = binary_cross_entropy_with_logits(pohp, heavy_precip_mask)
        tot_loss =  loss_estim + loss_detect + loss_detect_heavy

        opt = self.optimizers()
        learning_rate = opt.param_groups[0]['lr']
        
        self.log_dict(
            {
                "val_loss": loss_estim + loss_detect + loss_detect_heavy,
                "val_loss_estim": loss_estim,
                "val_loss_detect": loss_detect,
                "val_loss_detect_heavy": loss_detect_heavy,
                "learning_rate": learning_rate
            },
            on_epoch=True,
            prog_bar=True
        )
    
    def configure_optimizers(self) -> Dict[str, Any]:
        """
        We use the Adam optimizer with a cosine annealing learning rate schedule.
        """
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.n_epochs)
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler
        }

### Running the training

In [23]:
from ipwgml.input import calculate_input_features

input_features = calculate_input_features(inputs)
ipwg_unet = IPWGUNet(input_features=input_features, internal_features=[32, 64, 128, 256, 512], n_epochs=20)

In [24]:
trainer = L.Trainer(max_epochs=ipwg_unet.n_epochs)
trainer.fit(model=ipwg_unet, train_dataloaders=training_loader, val_dataloaders=validation_loader)

INFO: GPU available: True (cuda), used: True


[06/23/24 22:03:11] INFO     INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), ]8;id=313290;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=959767;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\
                             used: True                                                                            

INFO: TPU available: False, using: 0 TPU cores


                    INFO     INFO:lightning.pytorch.utilities.rank_zero:TPU available: False,       ]8;id=272093;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=9360;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\
                             using: 0 TPU cores                                                                    

INFO: IPU available: False, using: 0 IPUs


...

INFO: 
  | Name  | Type | Params
-------------------------------
0 | model | UNet | 8.4 M 
-------------------------------
8.4 M     Trainable params
0         Non-trainable params
8.4 M     Total params
33.408    Total estimated model params size (MB)


                    INFO     INFO:lightning.pytorch.callbacks.model_summary:                    ]8;id=707737;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_summary.py\model_summary.py]8;;\:]8;id=171847;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_summary.py#94\94]8;;\
                               | Name  | Type | Params                                                             
                             -------------------------------                                                       
                             0 | model | UNet | 8.4 M                                                              
                             -------------------------------                                                       
                             8.4 M     Trainable params                                                            
                             0         Non-trainable params                                                        
                             8.4 M     Total params                                                                
                             33.408    Total estimated model params size (MB)                                      

Sanity Checking: |                                                      | 0/? [00:00<?, ?it/s]

Training: |                                                             | 0/? [00:00<?, ?it/s]

KeyError: Caught KeyError in DataLoader worker process 21.
Original Traceback (most recent call last):
  File "/home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/xarray/core/dataset.py", line 1436, in _copy_listed
    variables[name] = self._variables[name]
KeyError: 'observations'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/hom

# Evaluating the retrieval

To evaluate the retrieval using the ``ipwgml.evaluation.Evaluator`` class, we need to define a retrieval callback function that performes inferences on the retrieval input data provided by the evaluator. The ``ipwgml.pytorch`` module provides the ``PytorchRetrieval`` helper class that wraps around a trained PyTorch neural network and acts as a ``retrieval_fn`` that can be passed to the evaluator.

In [37]:
from ipwgml.pytorch import PytorchRetrieval
unet_retrieval = PytorchRetrieval(
    ipwg_unet,
    retrieval_input=inputs,
    stack=True,
    device=torch.device("cuda"),
    dtype=torch.float32
)

In [38]:
from ipwgml.evaluation import Evaluator
evaluator = Evaluator(
    sensor="gmi",
    geometry=geometry,
    retrieval_input=inputs,
    download=True
)

[06/23/24 21:10:47] INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=527005;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=578865;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

                    INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=873655;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=397110;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

                    INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=5153;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=531446;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

                    INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=572319;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=162562;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

                    INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=658441;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=959030;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

                    INFO     INFO:requests_cache.backends.base:Clearing all items from the cache        ]8;id=882708;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py\base.py]8;;\:]8;id=521458;file:///home/simon/miniconda3/envs/ipwgml/lib/python3.10/site-packages/requests_cache/backends/base.py#108\108]8;;\

## Case study

## Running the evaluation 

In [39]:
evaluator.evaluate(
    retrieval_fn=unet_retrieval,
    input_data_format="spatial",
    batch_size=32,
    n_processes=1,
    tile_size=256,
    overlap=64,
    output_path="."
)

Output()

## Results

### Precipitation estimation

In [40]:
evaluator.get_precip_quantification_results(name="UNet (CPCIR)").T

/home/simon/src/ipwgml/src/ipwgml/metrics.py:645: RuntimeWarning: invalid value encountered in divide
  sigma_target = w_target_s2 / counts - (w_target_s / counts) ** 2
/home/simon/src/ipwgml/src/ipwgml/metrics.py:646: RuntimeWarning: invalid value encountered in divide
  sigma_pred = w_pred_s2 / counts - (w_pred_s / counts) ** 2
/home/simon/src/ipwgml/src/ipwgml/metrics.py:647: RuntimeWarning: invalid value encountered in divide
  target_mean = w_target_s / counts
/home/simon/src/ipwgml/src/ipwgml/metrics.py:648: RuntimeWarning: invalid value encountered in divide
  pred_mean = w_pred_s / counts
/home/simon/src/ipwgml/src/ipwgml/metrics.py:649: RuntimeWarning: invalid value encountered in divide
  targetpred_mean = w_targetpred_s / counts
/home/simon/src/ipwgml/src/ipwgml/metrics.py:654: RuntimeWarning: invalid value encountered in divide
  co = np.abs(w_targetpred_s) / (np.sqrt(w_target_s2) * np.sqrt(w_pred_s2))
/home/simon/src/ipwgml/src/ipwgml/metrics.py:672: RuntimeWarning: invali

,UNet (CPCIR),IMERG Final V7 (GMI),GPROF V7 (GMI)
Bias [$\%$],-15.526079,-7.398772,5.799770
MAE [$mm h^{-1}$],0.161556,0.124924,0.119785
MSE [$(mm h^{-1})^2$],1.845625,1.731411,1.413339
SMAPE$_{0.1}$ [$\%$],102.541505,114.417614,91.327371
Correlation coeff. [],0.493592,0.455411,0.552348
Effective resolution [$^\circ$],inf,inf,0.423000


### Precipitation detection

In [41]:
evaluator.get_precip_detection_results(name="UNet (CPCIR)").T

,UNet (CPCIR),IMERG Final V7 (GMI),GPROF V7 (GMI)
POD [],0.382926,0.687576,0.758366
FAR [],0.356477,0.497714,0.462010
HSS [],0.633071,0.470956,0.508161


### Probabilistic precipitation detection

In [42]:
evaluator.get_prob_precip_detection_results(name="UNet (CPCIR)").T

/home/simon/src/ipwgml/src/ipwgml/metrics.py:904: RuntimeWarning: invalid value encountered in divide
  precision = self.n_tp / (self.n_tp + self.n_fp)


,UNet (CPCIR),IMERG Final V7 (GMI),GPROF V7 (GMI)
AUC [],0.557225,0.0,0.54994


### Heavy precipitation detection

In [43]:
evaluator.get_heavy_precip_detection_results(name="UNet (CPCIR)").T

,UNet (CPCIR),IMERG Final V7 (GMI),GPROF V7 (GMI)
POD [],0.062328,0.687576,0.758366
FAR [],0.424756,0.497714,0.462010
HSS [],0.575115,0.470956,0.508161


### Probabilistic heavy precipitation detection

In [47]:
evaluator.get_prob_heavy_precip_detection_results(name="UNet (CPCIR)").T

/home/simon/src/ipwgml/src/ipwgml/metrics.py:904: RuntimeWarning: invalid value encountered in divide
  precision = self.n_tp / (self.n_tp + self.n_fp)


,UNet (CPCIR),IMERG Final V7 (GMI),GPROF V7 (GMI)
AUC [],0.245186,0.0,0.298044
